<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-hw2/blob/main/BLM6114_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hesaplamalı Anlambilim Dersi 2.Ödevi
Konusu: Sınıflandırma

## Package installations

In [ ]:
# !pip install datasets # uncomment if necessary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

### Necessary imports

In [ ]:
from datasets import load_dataset
import random

## Dataset preperations
sst2 için eğitim kümesini 60K-->5K, test kümesini 9K-->1K düşürerek kullanınız.

cola için eğitim kümesini 8K-->5K, test kümesini değiştirmeden kullanınız.

In [ ]:
# SST2
sst2 = load_dataset("turkish-nlp-suite/TrGLUE", "sst2")
sst2_train = sst2['train'].shuffle(seed=42).select(range(5000))  # Eğitim: 60K -> 5K
sst2_test = sst2['test'].shuffle(seed=42).select(range(1000))  # Test: 9K -> 1K

# CoLA
cola = load_dataset("turkish-nlp-suite/TrGLUE", "cola")
cola_train = cola['train'].shuffle(seed=42).select(range(5000))  # Eğitim: 5K
cola_test = cola['test']  # Test: Tüm test kümesi



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

train--00000-of-00001.jsonl:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

validation--00000-of-00001.jsonl:   0%|          | 0.00/3.46M [00:00<?, ?B/s]

test--00000-of-00001.jsonl:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

...

validation--00000-of-00001.jsonl:   0%|          | 0.00/131k [00:00<?, ?B/s]

test--00000-of-00001.jsonl:   0%|          | 0.00/130k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7916 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Check imported data

In [10]:
sst_train_len = len(sst2_train)
sst_test_len = len(sst2_test)
cola_train_len = len(cola_train)
cola_test_len = len(cola_test)

print(f"SST2 eğitim kümesi uzunluğu: {sst_train_len}")
print(f"SST2 test kümesi uzunluğu: {sst_test_len}")
print(f"CoLA eğitim kümesi uzunluğu: {cola_train_len}")
print(f"CoLA test kümesi uzunluğu: {cola_test_len}")

# sst2_train verisinin ilk 10 satırını yazdır
print("sst2_train:")
for i in range(10):
    print(sst2_train[i])

# sst2_test verisinin ilk 10 satırını yazdır
print("\nsst2_test:")
for i in range(10):
    print(sst2_test[i])

# cola_train verisinin ilk 10 satırını yazdır
print("\ncola_train:")
for i in range(10):
    print(cola_train[i])

# cola_test verisinin ilk 10 satırını yazdır
print("\ncola_test:")
for i in range(10):
    print(cola_test[i])

SST2 eğitim kümesi uzunluğu: 5000
SST2 test kümesi uzunluğu: 1000
CoLA eğitim kümesi uzunluğu: 5000
CoLA test kümesi uzunluğu: 1000
sst2_train:
{'sentence': 'film başladığında bir müddet izlemesem mi acaba, sıkıcı bir filme benziyor diye düşündüm. Ancak ilerledikçe iyi ki devam etmişim dedim. Farklı ve gerçekten iyi bir film. İyi seyirler..', 'label': 1}
{'sentence': 'Film gerçek hayata göndermeler yapıyor. Filmi çok başarılı buldum gerçekten ender güzel yapımlardan. Her sahnesi özenle hazırlanmış. Oyunculuklar, replikler harika, fazla söze gerek yok mutlaka izlenmesi gereken bir yapım, sinema severlere duyrulur :)', 'label': 1}
{'sentence': 'harika bir animasyon. çok eğlenceli. tıpkı birincisi gibi.', 'label': 1}
{'sentence': "Ümit ünal filmlerini genelde beğenirim bu filmde keyifle izlediğim filmlerden hele ki evdeki ilk ayaküstü sorgulama sahnesi çok bizden kara mizah ama gerilimi daha çok mizahı daha az olsaydı diye de ummadım değil... Demet Evgar'ın katmanlı karakterini izlemek he